# DSC212 — Recursive Spectral Modularity Partitioning

This notebook implements recursive spectral modularity partitioning (leading eigenvector of the modularity matrix) on Zachary's Karate Club network. It saves snapshot visualizations after each accepted split and records per-node metric evolution (degree centrality, betweenness, closeness, clustering coefficient) across iterations.

How to use:
- Install required packages if needed (networkx, numpy, scipy, pandas, matplotlib, seaborn).
- Run all cells. Outputs (figures + CSVs) will be written to an `outputs/` folder in the notebook directory.
- This notebook includes the helpers.py content and also writes it to `src/helpers.py` so you can push the file to your repo directly if you save the notebook outputs.

In [ ]:
# Uncomment to install requirements in an environment without them
# !pip install networkx numpy scipy pandas matplotlib seaborn jupyter jupytext

In [ ]:
%matplotlib inline
import os
import sys
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
import scipy.sparse.linalg as spla

sns.set(style="whitegrid")
OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)


In [ ]:
# Write helpers.py into src/helpers.py so the helper module is included in the notebook outputs
os.makedirs('src', exist_ok=True)
helpers_code = '''import numpy as np
import scipy.sparse as sp
import scipy.sparse.linalg as spla
import networkx as nx

def modularity_matrix_for_nodes(G, nodes, total_edges):
    """
    Build the modularity matrix B restricted to the given node list.
    B_ij = A_ij - (k_i * k_j) / (2m)
    degrees k_i and k_j are taken with respect to the whole graph G.
    total_edges is m (number of edges).
    Returns dense numpy array of shape (n, n)
    """
    idx = {n: i for i, n in enumerate(nodes)}
    n = len(nodes)
    A = np.zeros((n, n), dtype=float)
    degrees = np.array([G.degree(n) for n in nodes], dtype=float)
    # adjacency
    for u in nodes:
        for v in G[u]:
            if v in idx:
                A[idx[u], idx[v]] = 1.0
    k_outer = np.outer(degrees, degrees)
    B = A - k_outer / (2.0 * total_edges)
    return B


def leading_eigvec(B, which='LA', tol=1e-6):
    """
    Compute leading eigenvector (largest eigenvalue) of symmetric matrix B.
    Uses scipy.sparse.linalg.eigsh first, falls back to dense.
    Returns leading eigenvalue and eigenvector (numpy arrays).
    """
    n = B.shape[0]
    if n == 0:
        return 0.0, np.array([])
    if n == 1:
        return B[0,0], np.array([1.0])
    try:
        vals, vecs = spla.eigsh(sp.csr_matrix(B), k=1, which=which, tol=tol, maxiter=1000)
        return float(vals[0]), vecs[:,0]
    except Exception:
        vals, vecs = np.linalg.eigh(B)
        idx = np.argmax(vals)
        return float(vals[idx]), vecs[:, idx]


def partition_by_leading_eig(G, nodes, m):
    """
    Given graph G and a list of nodes (community), compute bipartition using leading eigenvector.
    Returns:
      - partA (list), partB (list), delta_Q (float), s vector (numpy array of +/-1)
    """
    B = modularity_matrix_for_nodes(G, nodes, total_edges=m)
    eigval, eigvec = leading_eigvec(B)
    if eigvec.size == 0:
        return [], [], 0.0, np.array([])
    # Partition by sign of leading eigenvector
    s = np.where(eigvec >= 0, 1, -1).astype(float)
    # If all same sign, then no split
    if np.all(s == 1) or np.all(s == -1):
        return [], [], 0.0, s
    # compute modularity change: (1/(4m)) * s^T B s
    delta_Q = float((s @ B @ s) / (4.0 * m))
    idx = {i: n for i, n in enumerate(nodes)}
    partA = [idx[i] for i in range(len(nodes)) if s[i] == 1]
    partB = [idx[i] for i in range(len(nodes)) if s[i] == -1]
    return partA, partB, delta_Q, s


def recursive_spectral_partition(G, stop_min_size=1):
    """
    Recursively partition the graph's nodes using spectral modularity splits.
    Accept splits only when delta_Q > 0 and both parts are non-empty and size >= stop_min_size.

    Returns:
      - communities: list of lists (final communities)
      - snapshots: list of dicts mapping node -> community_id after each accepted split (iteration snapshots)
    """
    m = G.number_of_edges()
    # community assignment dict: node -> community_id
    comm_assign = {n: 0 for n in G.nodes()}
    # queue of (community_id, node_list)
    next_comm_id = 1
    work = [(0, list(G.nodes()))]
    snapshots = []
    communities = {0: list(G.nodes())}

    while work:
        cid, nodes = work.pop(0)
        if len(nodes) <= stop_min_size:
            continue
        partA, partB, delta_Q, s = partition_by_leading_eig(G, nodes, m)
        if delta_Q > 1e-12 and len(partA) > 0 and len(partB) > 0:
            # accept split: assign new ids
            idA = cid  # reuse current id for one side
            idB = next_comm_id
            next_comm_id += 1
            # update communities dict
            communities[idA] = partA
            communities[idB] = partB
            # update assignments
            for n in partA:
                comm_assign[n] = idA
            for n in partB:
                comm_assign[n] = idB
            # update work queue for further splitting
            work.append((idA, partA))
            work.append((idB, partB))
            # snapshot after this accepted split (copy)
            snapshots.append(dict(comm_assign))
        else:
            # no split; leave community as is
            communities[cid] = nodes
            continue

    # produce final communities list
    # compress communities dict to unique sets
    seen = set()
    final = []
    for cid, nodes in communities.items():
        if cid in seen:
            continue
        if nodes:
            final.append(nodes)
            seen.update(nodes)
    return final, snapshots
'''

with open('src/helpers.py', 'w') as f:
    f.write(helpers_code)

# also create an __init__.py so `src` is recognized as a package
with open('src/__init__.py', 'w') as f:
    f.write('# package init for src')

print('Wrote src/helpers.py and src/__init__.py')


In [ ]:
def modularity_matrix_for_nodes(G, nodes, total_edges):
    """
    Build modularity matrix B restricted to `nodes`.
    B_ij = A_ij - k_i*k_j/(2m) where degrees are computed on the whole graph G.
    Returns numpy.ndarray (n x n).
    """
    idx = {n: i for i, n in enumerate(nodes)}
    n = len(nodes)
    if n == 0:
        return np.zeros((0,0))
    A = np.zeros((n, n), dtype=float)
    degrees = np.array([G.degree(n) for n in nodes], dtype=float)
    for u in nodes:
        for v in G[u]:
            if v in idx:
                A[idx[u], idx[v]] = 1.0
    k_outer = np.outer(degrees, degrees)
    B = A - k_outer / (2.0 * total_edges)
    return B

def leading_eigvec(B, which='LA', tol=1e-6):
    """
    Compute the leading eigenvalue and eigenvector of symmetric matrix B.
    Uses sparse ARPACK via eigsh when possible, falls back to dense eigh.
    """
    n = B.shape[0]
    if n == 0:
        return 0.0, np.array([])
    if n == 1:
        return float(B[0,0]), np.array([1.0])
    try:
        vals, vecs = spla.eigsh(sp.csr_matrix(B), k=1, which=which, tol=tol, maxiter=1000)
        return float(vals[0]), vecs[:, 0]
    except Exception:
        vals, vecs = np.linalg.eigh(B)
        idx = np.argmax(vals)
        return float(vals[idx]), vecs[:, idx]

def partition_by_leading_eig(G, nodes, m):
    """
    Compute bipartition of `nodes` by the sign of leading eigenvector of modularity matrix.
    Returns (partA, partB, delta_Q, s)
    s is array of +/-1 in the order of `nodes`.
    """
    B = modularity_matrix_for_nodes(G, nodes, total_edges=m)
    eigval, eigvec = leading_eigvec(B)
    if eigvec.size == 0:
        return [], [], 0.0, np.array([])
    s = np.where(eigvec >= 0, 1.0, -1.0)
    # if all same sign -> no split
    if np.all(s == 1.0) or np.all(s == -1.0):
        return [], [], 0.0, s
    delta_Q = float((s @ B @ s) / (4.0 * m))
    idx = {i: n for i, n in enumerate(nodes)}
    partA = [idx[i] for i in range(len(nodes)) if s[i] == 1.0]
    partB = [idx[i] for i in range(len(nodes)) if s[i] == -1.0]
    return partA, partB, delta_Q, s

def recursive_spectral_partition(G, stop_min_size=1):
    """
    Recursively partition G using spectral modularity splits.
    Accept splits only when delta_Q > 0 and both parts non-empty and meet stop_min_size.
    Returns final communities (list of node lists) and snapshots (list of node->comm_id dicts) after each accepted split.
    """
    m = G.number_of_edges()
    comm_assign = {n: 0 for n in G.nodes()}
    work = [(0, list(G.nodes()))]
    next_comm_id = 1
    snapshots = []
    communities = {0: list(G.nodes())}

    while work:
        cid, nodes = work.pop(0)
        if len(nodes) <= stop_min_size:
            continue
        partA, partB, delta_Q, s = partition_by_leading_eig(G, nodes, m)
        if delta_Q > 1e-12 and len(partA) > 0 and len(partB) > 0:
            idA = cid
            idB = next_comm_id
            next_comm_id += 1
            communities[idA] = partA
            communities[idB] = partB
            for n in partA:
                comm_assign[n] = idA
            for n in partB:
                comm_assign[n] = idB
            work.append((idA, partA))
            work.append((idB, partB))
            snapshots.append(dict(comm_assign))
        else:
            communities[cid] = nodes

    seen_nodes = set()
    final = []
    for nodes in communities.values():
        if not nodes:
            continue
        nset = tuple(sorted(nodes))
        if not set(nset).issubset(seen_nodes):
            final.append(list(nset))
            seen_nodes.update(nset)
    return final, snapshots


In [ ]:
def draw_community_graph(G, comm_assign, title=None, fname=None, seed=42):
    plt.figure(figsize=(8,6))
    pos = nx.spring_layout(G, seed=seed)
    comms = sorted(set(comm_assign.values()))
    pal = sns.color_palette("tab10", n_colors=max(3, len(comms)))
    color_map = {c: pal[i % len(pal)] for i, c in enumerate(comms)}
    node_colors = [color_map[comm_assign[n]] for n in G.nodes()]
    nx.draw_networkx_nodes(G, pos, node_size=300, node_color=node_colors)
    nx.draw_networkx_edges(G, pos, alpha=0.6)
    nx.draw_networkx_labels(G, pos)
    if title:
        plt.title(title)
    plt.axis('off')
    if fname:
        plt.savefig(fname, bbox_inches='tight', dpi=150)
    plt.show()
    plt.close()

def compute_node_metrics(G):
    deg_c = nx.degree_centrality(G)
    betw = nx.betweenness_centrality(G, normalized=True)
    close = nx.closeness_centrality(G)
    clust = nx.clustering(G)
    df = pd.DataFrame({
        "degree_centrality": pd.Series(deg_c),
        "betweenness_centrality": pd.Series(betw),
        "closeness_centrality": pd.Series(close),
        "clustering": pd.Series(clust),
    })
    return df


In [ ]:
def run_analysis(stop_min_size=1, save_plots=True):
    G = nx.karate_club_graph()
    m = G.number_of_edges()
    print(f"Loaded Karate graph: nodes={G.number_of_nodes()}, edges={m}")

    # initial snapshot (iteration 0)
    metrics0 = compute_node_metrics(G)
    metrics0.to_csv(os.path.join(OUTPUT_DIR, "metrics_iteration_0.csv"))
    init_assign = {n: 0 for n in G.nodes()}
    if save_plots:
        draw_community_graph(G, init_assign, title='Iteration 0 - initial community', fname=os.path.join(OUTPUT_DIR, 'graph_iter_0.png'))

    # recursive partitioning
    final_comms, snapshots = recursive_spectral_partition(G, stop_min_size=stop_min_size)
    # save final communities
    with open(os.path.join(OUTPUT_DIR, 'final_communities.txt'), 'w') as f:
        for i, c in enumerate(final_comms):
            f.write(f"Community {i}: {sorted(c)}\n")

    # collect metrics per iteration
    all_metrics = []
    m0 = metrics0.copy()
    m0['iteration'] = 0
    m0['community'] = m0.index.map(lambda n: 0)
    all_metrics.append(m0.reset_index().rename(columns={'index':'node'}))

    for it, snap in enumerate(snapshots, start=1):
        if save_plots:
            draw_community_graph(G, snap, title=f'Iteration {it}', fname=os.path.join(OUTPUT_DIR, f'graph_iter_{it}.png'))
        df = compute_node_metrics(G)
        df['iteration'] = it
        df['community'] = df.index.map(lambda n: snap.get(n, -1))
        all_metrics.append(df.reset_index().rename(columns={'index':'node'}))
        df.to_csv(os.path.join(OUTPUT_DIR, f'metrics_iteration_{it}.csv'))

    metrics_all = pd.concat(all_metrics, ignore_index=True)
    metrics_all.to_csv(os.path.join(OUTPUT_DIR, 'metrics_all_iterations.csv'), index=False)

    # Plot evolution lines per node (overlayed)
    fig, axes = plt.subplots(2,2, figsize=(12,10))
    for metric, ax in zip(["degree_centrality","betweenness_centrality","closeness_centrality","clustering"], axes.flatten()):
        sns.lineplot(data=metrics_all, x='iteration', y=metric, hue='node', estimator=None, alpha=0.6, legend=False, ax=ax)
        ax.set_title(f"{metric} evolution")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'metrics_evolution_lines.png'), dpi=150)
    plt.close()

    # Boxplots per iteration for each metric
    fig, axes = plt.subplots(2,2, figsize=(12,10))
    for metric, ax in zip(["degree_centrality","betweenness_centrality","closeness_centrality","clustering"], axes.flatten()):
        sns.boxplot(data=metrics_all, x='iteration', y=metric, ax=ax)
        ax.set_title(f"{metric} distribution by iteration")
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, 'metrics_evolution_boxplots.png'), dpi=150)
    plt.close()

    print('Saved outputs to', OUTPUT_DIR)
    for fname in sorted(os.listdir(OUTPUT_DIR)):
        print(' -', fname)

    return final_comms, snapshots, metrics_all

# Run the analysis
final_comms, snapshots, metrics_all = run_analysis(stop_min_size=1, save_plots=True)


## Notes and suggested next steps

- The recursion accepts a split only when it yields positive modularity gain (delta_Q > 0) and both sides are non-empty.
- You can adjust `stop_min_size` to require a minimum community size before attempting to split.
- The notebook now writes `src/helpers.py` (identical to the helper module used in the script) so you can extract that file directly from the notebook outputs if you need to push a separate helpers.py file to your repository.
- If you want, I can also produce a short Markdown report summarizing the discovered communities and notable node metric trends for submission.